In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
from pathlib import Path

# Configurar caminhos do projeto
ROOT_DIR = Path().absolute()
DOWNLOADS_DIR = ROOT_DIR / 'download'
DOWNLOADS_DIR.mkdir(parents=True, exist_ok=True)
print(f"Downloads serão salvos em: {DOWNLOADS_DIR}")

# Configurar o Chrome
options = Options()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36")

# Configurar pasta de download
prefs = {
    "download.default_directory": str(DOWNLOADS_DIR.absolute()),
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
options.add_experimental_option("prefs", prefs)

servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico, options=options)

Downloads serão salvos em: c:\Meus Documentos - Natasha\Desktop\pc\Documents\Projetos\api-caixa-imoveis\download


### Abrir o navegador e baixar tabela

In [2]:
navegador.get('https://venda-imoveis.caixa.gov.br/sistema/download-lista.asp')
time.sleep(3)

navegador.find_element(By.XPATH, '//*[@id="cmb_estado"]').click()
navegador.find_element(By.XPATH, '//*[@id="cmb_estado"]/option[2]').click()
navegador.find_element(By.ID, 'btn_next1').click()
time.sleep(10)

### Importando e exibindo a tabela

O arquivo CSV da Caixa provavelmente está em uma codificação Windows (como latin1 ou cp1252). 
Vamos tentar detectar a codificação correta e ler o arquivo usando o pandas.

In [3]:
# Encontrar CSV mais recente e carregar com pandas
try:
    # Pegar arquivo mais recente
    arquivo_csv = max(DOWNLOADS_DIR.glob("*.csv"), key=lambda p: p.stat().st_mtime)
    print(f"Arquivo encontrado: {arquivo_csv.name} - Data: {time.strftime('%d/%m/%y', time.localtime(arquivo_csv.stat().st_mtime))}")
    
    # Carregar com pandas
    tabela_imoveis = pd.read_csv(arquivo_csv, encoding='latin1', sep=';', header=1)
    
    # Remover espaços em branco de todas as colunas string
    colunas_string = tabela_imoveis.select_dtypes(include=['object']).columns
    for coluna in colunas_string:
        tabela_imoveis[coluna] = tabela_imoveis[coluna].str.rstrip()
    print("\nColunas processadas para remover espaços em branco:", list(colunas_string))
    
    # Renomear colunas
    tabela_imoveis = tabela_imoveis.rename(columns={' N° do imóvel': 'id', 'UF': 'uf', 'Cidade': 'cidade', 'Bairro': 'bairro', 'Endereço': 'endereco', 'Preço': 'preco', 'Valor de avaliação': 'valor', 'Desconto': 'desconto', 'Descrição': 'descricao', 'Modalidade de venda': 'venda', 'Link de acesso': 'link'})

    display(tabela_imoveis.head())
    
except ValueError as e:
    print(f"Nenhum arquivo CSV encontrado em {DOWNLOADS_DIR}")

Arquivo encontrado: Lista_imoveis_geral (3).csv - Data: 04/11/25

Colunas processadas para remover espaços em branco: ['UF', 'Cidade', 'Bairro', 'Endereço', 'Preço', 'Valor de avaliação', 'Descrição', 'Modalidade de venda', 'Link de acesso']


,id,uf,cidade,bairro,endereco,preco,valor,desconto,descricao,venda,link
0,1444419970935,AC,CRUZEIRO DO SUL,ZONA RURAL,"ESTRADA DO CANELA FINA, N. SN, PARTE 19, DESME...","99.222,38","250.000,00",60.32,"Terreno, 0.00 de área total, 0.00 de área priv...",Venda Direta Online,https://venda-imoveis.caixa.gov.br/sistema/det...
1,10005120,AC,CRUZEIRO DO SUL,BAIRRO COHAB,"RUA DO PURUS, N. S/N, QUARTEIRÃO 272, LOTE 10","71.534,67","150.000,00",52.32,"Terreno, 0.00 de área total, 0.00 de área priv...",Licitação Aberta,https://venda-imoveis.caixa.gov.br/sistema/det...
2,10005121,AC,CRUZEIRO DO SUL,BAIRRO COHAB,"RUA DO PURUS, N. S/N, QUARTEIRÃO 272 LOTE 11","71.534,67","150.000,00",52.32,"Terreno, 0.00 de área total, 0.00 de área priv...",Venda Direta Online,https://venda-imoveis.caixa.gov.br/sistema/det...
3,10005122,AC,CRUZEIRO DO SUL,BAIRRO COHAB,"RUA DO PURUS, N. S/N, QUARTEIRÃO 272 LOTE 12","71.534,67","150.000,00",52.32,"Terreno, 0.00 de área total, 0.00 de área priv...",Venda Direta Online,https://venda-imoveis.caixa.gov.br/sistema/det...
4,1444415405230,AC,CRUZEIRO DO SUL,Bairro AABB,"AVENIDA VITORIA REGIA, N. 0, Apto ., 1027","150.326,51","255.000,00",41.05,"Casa, 0.00 de área total, 131.20 de área priva...",Venda Direta Online,https://venda-imoveis.caixa.gov.br/sistema/det...


### Exportar pra Json

In [4]:
import json
from pathlib import Path

# pasta de saída no repositório

out_dir = ROOT_DIR / "Output"
out_dir.mkdir(parents=True, exist_ok=True)
out_file = out_dir / "index.json"

# Se tiver colunas datetime, converta para string no formato desejado (opcional)
# Ex.: tabela_imoveis['data'] = pd.to_datetime(tabela_imoveis['data']).dt.strftime('%d/%m/%y')

data = time.strftime('%d/%m/%y', time.localtime(arquivo_csv.stat().st_mtime))
# Substituir NaN por None ou string vazia, se quiser
records = tabela_imoveis.where(pd.notnull(tabela_imoveis), None).to_dict(orient="records")
resultado = {
    'metadata': {
        'date': data
    },
    'list': records
}

# Escrever JSON legível e com acentuação correta
with open(out_file, "w", encoding="utf-8") as f:
    json.dump(resultado, f, ensure_ascii=False, indent=2)

print(f"Exportado {len(records)} registros para {out_file}")

Exportado 32581 registros para c:\Meus Documentos - Natasha\Desktop\pc\Documents\Projetos\api-caixa-imoveis\Output\index.json
